In [1]:
from pydrake.all import (Meshcat, Simulator)
from manipulation.station import LoadScenario, MakeHardwareStation

from pydrake.planning import KinematicTrajectoryOptimization
from pydrake.solvers import Solve, SolverOptions, SnoptSolver
import numpy as np
import time

In [2]:
meshcat = Meshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
scenario_data = """
directives:
- add_model:
    name: mobile_iiwa
    file: package://manipulation/mobile_iiwa14_primitive_collision.urdf
model_drivers:
    mobile_iiwa: !InverseDynamicsDriver {}
"""

scenario = LoadScenario(data=scenario_data)
station = MakeHardwareStation(scenario, meshcat)
sim = Simulator(station)
context = sim.get_mutable_context()

x0 = station.GetOutputPort("mobile_iiwa.state_estimated").Eval(context)
station.GetInputPort("mobile_iiwa.desired_state").FixValue(context, x0)
sim.AdvanceTo(0.1)

In [4]:
plant = station.GetSubsystemByName("plant")
plant_context = plant.GetMyMutableContextFromRoot(context)

ix = plant.GetJointByName("iiwa_base_x").position_start()
iy = plant.GetJointByName("iiwa_base_y").position_start()
iz = plant.GetJointByName("iiwa_base_z").position_start()

q0  = plant.GetPositions(plant_context).copy()
qlo = plant.GetPositionLowerLimits()
qhi = plant.GetPositionUpperLimits()
vlo = plant.GetVelocityLowerLimits()
vhi = plant.GetVelocityUpperLimits()
nq  = plant.num_positions()

print("Number of positions: {}".format(nq))
print("Initial (x,y) position: {:.2f}, {:.2f}".format(q0[ix], q0[iy]))
print("Base x limits: [{:.2f}, {:.2f}]".format(qlo[ix], qhi[ix]))
print("Base y limits: [{:.2f}, {:.2f}]".format(qlo[iy], qhi[iy]))

Number of positions: 10
Initial (x,y) position: 0.00, 0.00
Base x limits: [-inf, inf]
Base y limits: [-inf, inf]


In [5]:
q_start = q0.copy()
q_goal  = q0.copy()
q_goal[ix] = q_start[ix] + 1
print("q_start:", q_start)
print("q_goal:", q_goal)

q_start: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
q_goal: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
vlo[[ix, iy, iz]] = -0.5
vhi[[ix, iy, iz]] = 0.5


In [7]:
N = 25
min_t = 0.5
max_t = 5.0
eps = 1e-6

kto = KinematicTrajectoryOptimization(nq, N)
prog = kto.get_mutable_prog()

kto.AddDurationCost(10.0)
kto.AddDurationConstraint(min_t, max_t)

kto.AddPositionBounds(qlo, qhi)
kto.AddVelocityBounds(vlo, vhi)

kto.AddPathPositionConstraint(q_start-eps, q_start+eps, 0)
kto.AddPathPositionConstraint(q_goal-eps, q_goal+eps, 1)

In [8]:
solver = SnoptSolver()
opts = SolverOptions()
opts.SetOption(solver.id(), "minor feasibility tolerance", 1e-6)
res = solver.Solve(prog, solver_options=opts)
if not res.is_success():
	print("traj opt failed")
	print("infeasible constraints", res.GetInfeasibleConstraintNames(prog))
	print("traj_opt solver name", res.get_solver_id().name())

traj = kto.ReconstructTrajectory(res)

In [ ]:
T = traj.end_time()
Ts = np.linspace(0.0, T, 180)
for t in Ts:
    q = traj.value(t).ravel()
    plant.SetPositions(plant_context, q)
    station.ForcedPublish(context)
    time.sleep(max(T/len(Ts)/1.5, 0.01))

plant.SetPositions(plant_context, traj.value(T).ravel())
sim.AdvanceTo(sim.get_context().get_time() + 0.5)
print("Done.")

Done.
